The Employee table holds all employees including their managers. Every employee has an Id, and there is also a column for the manager Id. <br>
+------+----------+-----------+----------+ <br>
|Id |Name |Department |ManagerId | <br>
+------+----------+-----------+----------+ <br>
|101 |John |A |null | <br>
|102 |Dan |A |101 | <br>
|103 |James |A |101 | <br>
|104 |Amy |A |101 | <br>
|105 |Anne |A |101 | <br>
|106 |Ron |B |101 | <br>
+------+----------+-----------+----------+ <br>
**Given the Employee table, write a SQL query that finds out managers with at least 5 direct report. For the above table, your SQL query should return:** <br>
+-------+ <br>
| Name | <br>
+-------+ <br>
| John | <br>
+-------+ <br>
**Note**: No one would report to himself.

Creating dataset

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# schema = StructType([
#   StructField("Id", IntegerType(), True),
#   StructField("Name", StringType(), True),
#   StructField("Department", StringType(), True),
#   StructField("ManagerId", IntegerType(), True)
# ])
schema = "Id int, Name string, Department string, ManagerId int"
data = [(101, "John", "A", None), 
         (102, "Dan", "A", 101), 
         (103, "James", "A", 101), 
         (104, "Amy", "A", 101), 
         (105, "Anne", "C", 101), 
         (106, "Dany", "D", 102), 
         (107, "Jamy", "D", 102), 
         (108, "Amno", "C", 101), 
         (109, "Antony", "B", 102),
         (110, "Samy", "B", 103),
         (111, "Ronny", "B", 102)]

df = spark.createDataFrame(data = data, schema = schema)

df.display()

Id,Name,Department,ManagerId
101,John,A,null
102,Dan,A,101
103,James,A,101
104,Amy,A,101
105,Anne,C,101
106,Dany,D,102
107,Jamy,D,102
108,Amno,C,101
109,Antony,B,102
110,Samy,B,103


In [0]:
from pyspark.sql.functions import col, count

df_manager = (df.alias("df1")
              .join(df.alias("df2"), col("df1.ManagerId") == col("df2.Id"), "inner")
              .groupBy("df1.ManagerId")
              .agg(count("*").alias("reportee_count"))
              .filter("reportee_count >= 5")
              .select("ManagerId")
              )

df_manager_with_5reportee = (df_manager.alias("dfm")
                             .join(df.alias("dfe"), col("dfm.ManagerId") == col("dfe.Id"), "inner")
                             .select("dfe.Name"))

df_manager_with_5reportee.display()

Name
John
